In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import scipy as sp
from tqdm import auto, tqdm
import torch
from sklearn.model_selection import train_test_split

import sys
sys.path.append('/content/drive/MyDrive/11777-Project-xs/LGG/')
from prepare_data_DEAP import PrepareData
from cross_validation import CrossValidation
from train_model import *

In [3]:
original_order = ['Fp1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'P3', 'P7', 'PO3', 'O1', 'Oz',
                  'Pz', 'Fp2', 'AF4', 'Fz', 'F4', 'F8', 'FC6', 'FC2', 'Cz', 'C4', 'T8', 'CP6', 'CP2', 'P4', 'P8',
                  'PO4', 'O2']

# Define proper channel orders for the local-global graphs in LGGNet. Please refer to three graph definitions (general, frontal, hemesphere).
graph_general_DEAP = [['Fp1', 'Fp2'], ['AF3', 'AF4'], ['F3', 'F7', 'Fz', 'F4', 'F8'],
                     ['FC5', 'FC1', 'FC6', 'FC2'], ['C3', 'Cz', 'C4'], ['CP5', 'CP1', 'CP2', 'CP6'],
                     ['P7', 'P3', 'Pz', 'P4', 'P8'], ['PO3', 'PO4'], ['O1', 'Oz', 'O2'],
                     ['T7'], ['T8']]

graph_idx = graph_general_DEAP   # The general graph definition for DEAP is used as an example.
idx = []
num_chan_local_graph = []
for i in range(len(graph_idx)):
    num_chan_local_graph.append(len(graph_idx[i]))
    for chan in graph_idx[i]:
        idx.append(original_order.index(chan))

LGG = LGGNet(
    num_classes=2,
    input_size=(1, 32, 512),
    sampling_rate=128,
    num_T=64,  # num_T controls the number of temporal filters
    out_graph=32,
    pool=16,
    pool_step_rate=0.25,
    idx_graph=num_chan_local_graph,
    dropout_rate=0.5
)

LGG

In [55]:
# Define default parameters
default_args = {
    'dataset': 'DEAP',
    'data_path': './data_preprocessed_python/',
    'subjects': 22,
    'num_class': 2,
    'label_type': 'V',
    'segment': 4,
    'overlap': 0,
    'sampling_rate': 128,
    'scale_coefficient': 1,
    'input_shape': (1, 32, 512),
    'data_format': 'eeg',
    'random_seed': 2021,
    'max_epoch': 50, #200
    'patient': 5, #20,
    'patient_cmb': 8,
    'max_epoch_cmb': 20,
    'batch_size': 64,
    'learning_rate': 1e-3,
    'step_size': 5,
    'dropout': 0.5,
    'LS': True,
    'LS_rate': 0.1,
    'save_path': '/content/drive/MyDrive/11777-Project-xs/LGG/save/',
    'load_path': '/content/drive/MyDrive/11777-Project-xs/LGG/save/max-acc.pth',
    'load_path_final': '/content/drive/MyDrive/11777-Project-xs/LGG/save/final_model.pth',
    'mlp_save_path': '/content/drive/MyDrive/11777-Project-xs/MLP/save/',
    'mlp_load_path': '/content/drive/MyDrive/11777-Project-xs/MLP/save/max-acc.pth',
    'mlp_load_path_final': '/content/drive/MyDrive/11777-Project-xs/MLP/save/final_model.pth',
    'gpu': '0',
    'save_model': True,
    'model': 'LGGNet',
    'pool': 16,
    'pool_step_rate': 0.25,
    'T': 64,
    'graph_type': 'hem',
    'hidden': 32
}

In [59]:
######## Reproduce the result using the saved model ######
# parser.add_argument('--reproduce', action='store_true')
# args = parser.parse_args()
class Args:
    def __init__(self, param_dict):
        for key, value in param_dict.items():
            setattr(self, key, value)

default_args['data_path'] = '/content/drive/MyDrive/11777-Project-xs/data_preprocessed_python/'
default_args['reproduce'] = 'store_true'
# Create an Args object with default parameters
args = Args(default_args)


In [11]:
# only need to run once
sub_to_run = np.arange(args.subjects)
prepdt = PrepareData(args)
prepdt.run(sub_to_run, split=True, expand=True)

data:(40, 32, 7680) label:(40, 4)
Binary label generated!
The data and label are split: Data shape:(40, 15, 1, 32, 512) Label:(40, 15)
Data and label prepared!
data:(40, 15, 1, 32, 512) label:(40, 15)
----------------------
data:(40, 32, 7680) label:(40, 4)
Binary label generated!
The data and label are split: Data shape:(40, 15, 1, 32, 512) Label:(40, 15)
Data and label prepared!
data:(40, 15, 1, 32, 512) label:(40, 15)
----------------------
data:(40, 32, 7680) label:(40, 4)
Binary label generated!
The data and label are split: Data shape:(40, 15, 1, 32, 512) Label:(40, 15)
Data and label prepared!
data:(40, 15, 1, 32, 512) label:(40, 15)
----------------------
data:(40, 32, 7680) label:(40, 4)
Binary label generated!
The data and label are split: Data shape:(40, 15, 1, 32, 512) Label:(40, 15)
Data and label prepared!
data:(40, 15, 1, 32, 512) label:(40, 15)
----------------------
data:(40, 32, 7680) label:(40, 4)
Binary label generated!
The data and label are split: Data shape:(40, 

In [12]:
participant_ids = list(range(1, 23))
data_save_folder = '/content/drive/MyDrive/11777-Project-xs/data_face_seg_mean'

def delete_file(path):
    '''
    used to remove temporal file generated
    '''
    if os.path.exists(path):
        os.remove(path)

# # only need to run once
# for participant_id in auto.tqdm(participant_ids):
#     participant_id = '{:02d}'.format(participant_id)
#     for trial_id in auto.tqdm(range(1, 41)):
#         file_name = 's{0}_trial{1}_mean_frames.npy'.format(participant_id, trial_id)
#         if not os.path.exists(os.path.join(data_save_folder, file_name)):
#             print('No face data found for s{0}_trial{1}'.format(participant_id, trial_id))
#             continue

#         file_name2 = 's{0}_trial{1}_mean_frames_embed.npy'.format(participant_id, trial_id)
#         if os.path.exists(os.path.join(data_save_folder, file_name2)):
#             continue
#         participant_trial_seg = np.load(os.path.join(data_save_folder, file_name))
#         participant_trial_embed = []
#         for image in participant_trial_seg:
#             delete_file('curr_face_image.png')
#             plt.imsave('curr_face_image.png', np.uint8(participant_trial_seg[0]),
#                        cmap=plt.cm.Spectral)
#             embedding_objs = DeepFace.represent(img_path = 'curr_face_image.png', enforce_detection = False)
#             embedding = embedding_objs[0]['embedding']
#             assert isinstance(embedding, list)
#             assert len(embedding) == 2622
#             participant_trial_embed.append(np.array(embedding))

#         participant_trial_embed = np.stack(participant_trial_embed, axis=0)
#         np.save(os.path.join(data_save_folder, file_name2), participant_trial_embed)


In [13]:
import os
data_save_folder = '/content/drive/MyDrive/11777-Project-xs/data_face_seg_mean'
face_embed_file_list = os.listdir(data_save_folder)
# print(len(face_embed_file_list))
face_embed_file_list = [e for e in face_embed_file_list if 'embed.npy' in e]
sub_trial_list = {}
for file in face_embed_file_list:
    # print(file)
    sub, trial, _, _, _ = file.split('_')
    sub, trial = int(sub[1:]), int(trial[5:])
    if sub in sub_trial_list.keys():
        sub_trial_list[sub].append(trial)
    else:
        sub_trial_list[sub] = [trial]

for sub in range(1,23):
    print('sub {}, {} trials with facial video'.format(sub, len(sub_trial_list[sub])))

sub 1, 40 trials with facial video
sub 2, 40 trials with facial video
sub 3, 39 trials with facial video
sub 4, 40 trials with facial video
sub 5, 39 trials with facial video
sub 6, 40 trials with facial video
sub 7, 40 trials with facial video
sub 8, 40 trials with facial video
sub 9, 40 trials with facial video
sub 10, 40 trials with facial video
sub 11, 37 trials with facial video
sub 12, 40 trials with facial video
sub 13, 40 trials with facial video
sub 14, 39 trials with facial video
sub 15, 40 trials with facial video
sub 16, 40 trials with facial video
sub 17, 40 trials with facial video
sub 18, 40 trials with facial video
sub 19, 40 trials with facial video
sub 20, 40 trials with facial video
sub 21, 40 trials with facial video
sub 22, 40 trials with facial video


In [10]:
def load_mean_frames_embed(sub, data_save_folder):
    '''
    sub starting index from 1 (not 0)
    '''
    face_embed_file_list = os.listdir(data_save_folder)
    sub_face_embed_file_list = [e for e in face_embed_file_list if 'embed' in e and sub==int(e[1:3])]
    sorted(sub_face_embed_file_list)
    # print(sub_face_embed_file_list)
    sub_frame_embed = []
    for file in auto.tqdm(sub_face_embed_file_list):
        embed = np.load(os.path.join(data_save_folder, file))
        sub_frame_embed.append(embed)
    sub_frame_embed = np.stack(sub_frame_embed, axis=0)
    print(sub_frame_embed.shape)
    return sub_frame_embed

# 's01_trial1_mean_frames_embed.npy'

data_save_folder = '/content/drive/MyDrive/11777-Project-xs/data_face_seg_mean'
all_sub_frame_embed = []
for sub in auto.tqdm(range(1,23)):
    sub_frame_embed = load_mean_frames_embed(sub, data_save_folder)
    all_sub_frame_embed.append(sub_frame_embed)


  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

(40, 15, 2622)


  0%|          | 0/40 [00:00<?, ?it/s]

(40, 15, 2622)


...

(40, 15, 2622)


  0%|          | 0/40 [00:00<?, ?it/s]

(40, 15, 2622)


  0%|          | 0/40 [00:00<?, ?it/s]

(40, 15, 2622)


In [14]:
import pickle
file_name = 'all_sub_frame_embed.pkl'
# with open(os.path.join(data_save_folder, file_name), 'wb') as file:
#     pickle.dump(all_sub_frame_embed, file)

with open(os.path.join(data_save_folder, file_name), 'rb') as file:
    all_sub_frame_embed = pickle.load(file)

In [48]:
def generate_eeg_embed(eeg_data, LGG, CUDA=True):
    LGG.eval()
    eeg_embed = []
    # eeg_data = np.concatenate(eeg_data, axis=0)
    for eeg_data_trial in eeg_data:
        eeg_data_trial = torch.from_numpy(eeg_data_trial).float()
        if CUDA:
            eeg_data_trial = eeg_data_trial.cuda()
            LGG = LGG.cuda()
        with torch.no_grad():
            eeg_embed_trial = LGG.get_embed(eeg_data_trial)
        eeg_embed_trial = eeg_embed_trial.to('cpu')
        eeg_embed_trial = eeg_embed_trial.numpy()
        eeg_embed.append(eeg_embed_trial)
    eeg_embed = np.stack(eeg_embed, axis=0)
    return eeg_embed

In [17]:
def get_model_embed(args):
    if args.model == 'LGGNet':
        idx_local_graph = list(np.array(h5py.File('num_chan_local_graph_{}.hdf'.format(args.graph_type), 'r')['data']))
        channels = sum(idx_local_graph)
        input_size = (args.input_shape[0], channels, args.input_shape[2])
        model = LGGNet2(
            num_classes=args.num_class, input_size=input_size,
            sampling_rate=int(args.sampling_rate*args.scale_coefficient),
            num_T=args.T, out_graph=args.hidden,
            dropout_rate=args.dropout,
            pool=args.pool, pool_step_rate=args.pool_step_rate,
            idx_graph=idx_local_graph)

    return model

def transfer_weight(LGG, LGG_embed):
    for i, (pre_trained_layer, custom_layer) in enumerate(zip(LGG.children(), LGG_embed.children())):
        custom_layer.load_state_dict(pre_trained_layer.state_dict())
    return LGG_embed

In [37]:
all_sub_frame_embed[sub].shape

(40, 15, 2622)

In [44]:
import torch
import torch.nn as nn

class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_prob=0.5):
        super(SimpleMLP, self).__init__()
        # Input layer
        self.input_layer = nn.Linear(input_dim, hidden_dim)
        # Hidden layer
        self.hidden_layer = nn.Linear(hidden_dim, hidden_dim)
        # Output layer
        self.output_layer = nn.Linear(hidden_dim, output_dim)
        # Dropout layer
        self.dropout = nn.Dropout(p=dropout_prob)

    def forward(self, x):
        # Input layer
        x = self.input_layer(x)
        x = torch.relu(x)

        # Hidden layer with dropout
        x = self.hidden_layer(x)
        x = torch.relu(x)
        x = self.dropout(x)

        # Output layer
        x = self.output_layer(x)
        return x


In [63]:
from utils import *
import torch.nn as nn

CUDA = torch.cuda.is_available()

def mlp_set_up(args):
    set_gpu(args.gpu)
    ensure_path(args.mlp_save_path)
    torch.manual_seed(args.random_seed)
    torch.backends.cudnn.deterministic = True

def train_simpleMLP(model, args, data_train, label_train, data_val, label_val, subject):
    seed_all(args.random_seed)
    save_name = '_sub' + str(subject) + '_mlp'
    mlp_set_up(args)

    train_loader = get_dataloader(data_train, label_train, args.batch_size)

    val_loader = get_dataloader(data_val, label_val, args.batch_size)

    # model = get_model(args)
    if CUDA:
        model = model.cuda()

    optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate)

    if args.LS:
        loss_fn = LabelSmoothing(args.LS_rate)
    else:
        loss_fn = nn.CrossEntropyLoss()


    def save_model(name):
        previous_model = osp.join(args.mlp_save_path, '{}.pth'.format(name))
        if os.path.exists(previous_model):
            os.remove(previous_model)
        torch.save(model.state_dict(), osp.join(args.mlp_save_path, '{}.pth'.format(name)))

    trlog = {}
    trlog['args'] = vars(args)
    trlog['train_loss'] = []
    trlog['val_loss'] = []
    trlog['train_acc'] = []
    trlog['val_acc'] = []
    trlog['max_acc'] = 0.0
    trlog['F1'] = 0.0

    timer = Timer()
    patient = args.patient
    counter = 0

    for epoch in range(1, args.max_epoch + 1):

        loss_train, pred_train, act_train = train_one_epoch(
            data_loader=train_loader, net=model, loss_fn=loss_fn, optimizer=optimizer)

        acc_train, f1_train, _ = get_metrics(y_pred=pred_train, y_true=act_train)
        print('epoch {}, loss={:.4f} acc={:.4f} f1={:.4f}'
              .format(epoch, loss_train, acc_train, f1_train))

        loss_val, pred_val, act_val = predict(
            data_loader=val_loader, net=model, loss_fn=loss_fn
        )
        acc_val, f1_val, _ = get_metrics(y_pred=pred_val, y_true=act_val)
        print('epoch {}, val, loss={:.4f} acc={:.4f} f1={:.4f}'.
              format(epoch, loss_val, acc_val, f1_val))

        if acc_val >= trlog['max_acc']:
            trlog['max_acc'] = acc_val
            trlog['F1'] = f1_val
            save_model('candidate')
            counter = 0
        else:
            counter += 1
            if counter >= patient:
                print('early stopping')
                break

        trlog['train_loss'].append(loss_train)
        trlog['train_acc'].append(acc_train)
        trlog['val_loss'].append(loss_val)
        trlog['val_acc'].append(acc_val)

        print('ETA:{}/{} SUB:{}'.format(timer.measure(), timer.measure(epoch / args.max_epoch),
                                                 subject))
    # save the training log file
    save_name = 'trlog' + save_name
    experiment_setting = 'T_{}_pool_{}'.format(args.T, args.pool)
    save_path = osp.join(args.mlp_save_path, experiment_setting, 'log_train')
    ensure_path(save_path)
    torch.save(trlog, osp.join(save_path, save_name))

    return trlog['max_acc'], trlog['F1']

In [64]:
def test_simpleMLP(model, args, data, label, reproduce, subject):
    set_up(args)
    seed_all(args.random_seed)
    test_loader = get_dataloader(data, label, args.batch_size)

    # model = get_model(args)
    if CUDA:
        model = model.cuda()
    loss_fn = nn.CrossEntropyLoss()

    if reproduce:
        model_name_reproduce = 'sub' + str(subject) + '_mlp' + '.pth'
        data_type = 'model_{}_{}'.format(args.data_format, args.label_type)
        experiment_setting = 'T_{}_pool_{}'.format(args.T, args.pool)
        load_path_final = osp.join(args.mlp_save_path, experiment_setting, data_type, model_name_reproduce)
        model.load_state_dict(torch.load(load_path_final))
    else:
        model.load_state_dict(torch.load(args.mlp_load_path_final))
    loss, pred, act = predict(
        data_loader=test_loader, net=model, loss_fn=loss_fn
    )
    acc, f1, cm = get_metrics(y_pred=pred, y_true=act)
    print('>>> Test:  loss={:.4f} acc={:.4f} f1={:.4f}'.format(loss, acc, f1))
    return acc, pred, act

In [71]:
CUDA = torch.cuda.is_available()
fold = 'noFold'

with open(os.path.join(data_save_folder, file_name), 'rb') as file:
    all_sub_frame_embed = pickle.load(file)

for sub in range(22):

    print('='*48)
    cv = CrossValidation(args)
    eeg_data, label = cv.load_per_subject(sub)
    trial_to_remove = [i-1 for i in range(1,41) if i not in sub_trial_list[sub+1]]
    ### face embeddings
    sub_frame_embed = all_sub_frame_embed[sub]

    if len(trial_to_remove) > 0:
        mask = np.ones(eeg_data.shape[0], dtype=bool)
        mask[trial_to_remove] = False
        eeg_data = eeg_data[mask]
        label = label[mask]

    ### Load pre-trained LGG model weights
    ### and generate the EEG embeddings
    lgg_model = get_model(args)
    lgg_model_embed = get_model_embed(args)
    if CUDA:
        lgg_model = lgg_model.cuda()
    model_name_reproduce = 'sub' + str(sub+1) + '.pth'
    data_type = 'model_{}_{}'.format(args.data_format, args.label_type)
    experiment_setting = 'T_{}_pool_{}'.format(args.T, args.pool)
    save_path = os.path.join(args.save_path, experiment_setting, data_type)
    ensure_path(save_path)
    model_name_reproduce = os.path.join(save_path, model_name_reproduce)
    lgg_model.load_state_dict(torch.load(model_name_reproduce))
    lgg_model_embed = transfer_weight(lgg_model, lgg_model_embed)
    eeg_embed = generate_eeg_embed(eeg_data, lgg_model_embed, CUDA)

    ### concat eeg + face embedding
    eeg_face_embed = np.concatenate([eeg_embed, sub_frame_embed], axis=2)

    print(sub+1, trial_to_remove, sub_frame_embed.shape[0])
    trial_id = np.arange(eeg_data.shape[0])
    trial_id_train, trial_id_test = train_test_split(trial_id, test_size=0.2, random_state=42)
    trial_id_tra, trial_id_val = train_test_split(trial_id_train, test_size=0.2, random_state=42)
    print(trial_id_tra.shape[0], trial_id_val.shape[0], trial_id_test.shape[0])

    data_train, data_val, label_train, label_val = \
        eeg_face_embed[trial_id_tra], eeg_face_embed[trial_id_val], label[trial_id_tra], label[trial_id_val]
    data_train, data_val, label_train, label_val = \
        np.concatenate(data_train, axis=0), np.concatenate(data_val, axis=0), np.concatenate(label_train, axis=0), np.concatenate(label_val, axis=0)
    print(data_train.shape, data_val.shape, label_train.shape, label_val.shape)

    data_test, label_test = \
        eeg_face_embed[trial_id_test], label[trial_id_test]
    data_test, label_test = \
        np.concatenate(data_test, axis=0), np.concatenate(label_test, axis=0)
    print(data_test.shape, label_test.shape)

    data_train = torch.from_numpy(data_train).float()
    label_train = torch.from_numpy(label_train).long()
    data_val = torch.from_numpy(data_val).float()
    label_val = torch.from_numpy(label_val).long()
    data_test = torch.from_numpy(data_test).float()
    label_test = torch.from_numpy(label_test).long()


    ### define a simple MLP as the predictor
    predictor = SimpleMLP(data_train.shape[1], data_train.shape[1] // 4, 2)

    acc_val, F1_val = train_simpleMLP(model=predictor, args=args,
                                      data_train=data_train,
                                      label_train=label_train,
                                      data_val=data_val,
                                      label_val=label_val,
                                      subject = str(sub+1))

    if CUDA:
        predictor = predictor.cuda()
    predictor.load_state_dict(torch.load(os.path.join(args.mlp_save_path, 'candidate.pth')))

    model_name_reproduce = 'sub' + str(sub+1) + '_mlp.pth'
    data_type = 'model_{}_{}'.format(args.data_format, args.label_type)
    experiment_setting = 'T_{}_pool_{}'.format(args.T, args.pool)
    save_path = os.path.join(args.mlp_save_path, experiment_setting, data_type)
    ensure_path(save_path)
    model_name_reproduce = os.path.join(save_path, model_name_reproduce)
    print(model_name_reproduce)
    torch.save(predictor.state_dict(), model_name_reproduce)

    acc_test, pred, act = test_simpleMLP(model=predictor, args=args, data=data_test, label=label_test,
                                               reproduce=args.reproduce,
                                               subject=str(sub+1))

>>> Data:(40, 15, 1, 32, 512) Label:(40, 15)
1 [] 40
25 7 8
(375, 3166) (105, 3166) (375,) (105,)
(120, 3166) (120,)
using gpu: 0
epoch 1, loss=0.7542 acc=0.5147 f1=0.3636
epoch 1, val, loss=0.7085 acc=0.4286 f1=0.6000
ETA:0s/4s SUB:1
epoch 2, loss=0.7127 acc=0.4960 f1=0.4960
epoch 2, val, loss=0.6825 acc=0.5714 f1=0.0000
ETA:0s/4s SUB:1
epoch 3, loss=0.6682 acc=0.6293 f1=0.6128
epoch 3, val, loss=0.6876 acc=0.5714 f1=0.2623
ETA:0s/4s SUB:1
epoch 4, loss=0.6423 acc=0.6667 f1=0.6313
epoch 4, val, loss=0.7070 acc=0.5714 f1=0.1509
ETA:0s/4s SUB:1
epoch 5, loss=0.6018 acc=0.7013 f1=0.6706
epoch 5, val, loss=0.7587 acc=0.5619 f1=0.0800
ETA:0s/4s SUB:1
epoch 6, loss=0.5499 acc=0.7573 f1=0.7284
epoch 6, val, loss=0.7576 acc=0.4762 f1=0.2667
ETA:0s/3s SUB:1
epoch 7, loss=0.5163 acc=0.7733 f1=0.7522
epoch 7, val, loss=0.9109 acc=0.4667 f1=0.5484
ETA:0s/3s SUB:1
epoch 8, loss=0.5150 acc=0.7573 f1=0.7560
epoch 8, val, loss=0.8936 acc=0.4952 f1=0.5138
ETA:0s/3s SUB:1
epoch 9, loss=0.4810 acc=0.789